# Tutorial: Elasticity
In this tutorial we demonstrate the use of the compute_elasticity and compute_stress_analysis functions. 

In [1]:
# ONLY FOR GOOGLE COLAB: Run this cell only the first time you open a tutorial
if 'google.colab' in str(get_ipython()):
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    !conda install -c fsemerar puma

Let's start by importing pumapy, Numpy and Scipy ndimage.

In [2]:
import os
import pumapy as puma
import numpy as np
import scipy.ndimage as nd
import pyvista as pv

Now, we will run four different verification cases. Change the path of the file outputs:

In [3]:
export_path = "../tests/out/"  # CHANGE THIS PATH

### Verification 1: harmonic averaging, in series along x periodic sides

In [4]:
export_name = 'halfmat'
X = 20
Y = 20
Z = 20
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws[int(X / 2):] = 2
# ws.show_matrix()

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)
elast_map.add_isotropic_material((2, 2), 400, 0.1)

_, u, s, t = puma.compute_elasticity(ws, elast_map, 'x', 'f', solver_type="direct")

Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Direct solver ... Done
Computing stresses ... 100.0% Done


In [5]:
s.shape

(20, 20, 20, 3)

In [6]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("Displacement in x")
puma.render_volume(u[:, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("Displacement in y")
puma.render_volume(u[:, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("Displacement in z")
puma.render_volume(u[:, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [7]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("XX stress")
puma.render_volume(s[:, :10, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("YY stress")
puma.render_volume(s[:, :10, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("ZZ stress")
puma.render_volume(s[:, :10, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [8]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("YZ stress")
puma.render_volume(t[:, :10, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("XZ stress")
puma.render_volume(t[:, :10, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("XY stress")
puma.render_volume(t[:, :10, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [9]:
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

Exporting ../tests/out/halfmat.vti ... Done


True

### Verification 2: full built-in beam

In [10]:
export_name = 'builtinbeam'
X = 10
Y = 50
Z = 10
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[:, 0] = 0
bc[:, -1, :, :2] = 0
bc[:, -1, :, 2] = -1  # z displacement of -1
# puma.Workspace.show_orientation(bc)

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Direct solver ... Done
Computing stresses ... 100.0% Done


In [11]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("Displacement in x")
puma.render_volume(u[:, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("Displacement in y")
puma.render_volume(u[:, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("Displacement in z")
puma.render_volume(u[:, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [12]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("XX stress")
puma.render_volume(s[:5, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("YY stress")
puma.render_volume(s[:5, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("ZZ stress")
puma.render_volume(s[:5, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [13]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("YZ stress")
puma.render_volume(t[:5, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("XZ stress")
puma.render_volume(t[:5, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("XY stress")
puma.render_volume(t[:5, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [14]:
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

Exporting ../tests/out/builtinbeam.vti ... Done


True

### Verification 3: plate with a hole

In [25]:
export_name = 'platehole'
X = 50
Y = 50
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

mask = np.ones((X, Y, Z), dtype=bool)
mask[X//2, Y//2] = 0
distance_mask = nd.morphology.distance_transform_edt(mask)
max_distance = np.max(distance_mask)
distance_mask_display = (distance_mask*255./max_distance).astype(dtype=np.uint8)
in_range = distance_mask <= 17
ws[in_range] = 0

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[0, :, :, 0] = 0
bc[-1, :, :, 0] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Direct solver ... Done
Computing stresses ... 100.0% Done


In [26]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("Displacement in x")
puma.render_volume(u[:, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("Displacement in y")
puma.render_volume(u[:, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("Displacement in z")
puma.render_volume(u[:, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [27]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("XX stress")
puma.render_volume(s[:, :, 1:, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("YY stress")
puma.render_volume(s[:, :, 1:, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("ZZ stress")
puma.render_volume(s[:, :, 1:, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [28]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("YZ stress")
puma.render_volume(t[:, :, 1:, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("XZ stress")
puma.render_volume(t[:, :, 1:, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("XY stress")
puma.render_volume(t[:, :, 1:, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [19]:
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

Exporting ../tests/out/platehole.vti ... Done


True

### Verification 4: cracked plate

In [20]:
export_name = 'crackedplate'
X = 25
Y = 100
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

ws[:10, Y//2-1:Y//2+1] = 0

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC.from_workspace(ws)
bc[:, 0, :, 1] = 0
bc[:, -1, :, 1] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

Initializing and padding domains ... Done
Assembling b vector ... Done
Initializing large data structures ... Done
Assembling A matrix ... 100.0% Done
Solving Ax=b system ... Direct solver

/Users/fsemerar/opt/anaconda3/envs/puma/lib/python3.7/site-packages/scikits/umfpack/umfpack.py:721: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 2.08e+12)
  warnings.warn(msg, UmfpackWarning)


 ... Done
Computing stresses ... 100.0% Done


In [21]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("Displacement in x")
puma.render_volume(u[:, :, :, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("Displacement in y")
puma.render_volume(u[:, :, :, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("Displacement in z")
puma.render_volume(u[:, :, :, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [22]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("XX stress")
puma.render_volume(s[:, :, 1:, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("YY stress")
puma.render_volume(s[:, :, 1:, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("ZZ stress")
puma.render_volume(s[:, :, 1:, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [23]:
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("YZ stress")
puma.render_volume(t[:, :, 1:, 0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("XZ stress")
puma.render_volume(t[:, :, 1:, 1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("XY stress")
puma.render_volume(t[:, :, 1:, 2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show(cpos="xy")

ViewInteractiveWidget(height=1200, layout=Layout(height='auto', width='100%'), width=1920)

In [24]:
puma.export_vti(export_path + export_name, {"ws": ws, "disp": u, "sigma": s, "tau": t})

Exporting ../tests/out/crackedplate.vti ... Done


True